In [ ]:
from manim import *
import textwrap
import numpy as np

class LayoutDeepDiveExtended(Scene):
    """
    Extended Layout Deep Dive.
    - Original: Groups, Arrange, Align_to.
    - NEW: Sizing/Stretch, Grids, Dynamic Groups, Anchors, Z-Index.
    - IMPROVED: Added visual cues (bounding boxes, alignment lines) and text wrapping.
    """
    def construct(self):
        self.camera.background_color = "#1a1a1a"
        
        # 1. SETUP SCAFFOLDING
        self.setup_scaffolding("Manim Layout Engine")

        # --- ORIGINAL SECTIONS ---
        self.explain_groups()
        self.clear_stage()

        self.explain_arrange()
        self.clear_stage()

        self.explain_alignment_matchers()
        self.clear_stage()

        # --- NEW SECTION 1: SIZING & DISTORTION ---
        self.explain_sizing_stretch()
        self.clear_stage()

        # --- NEW SECTION 2: GRIDS & DYNAMIC GROUPS ---
        self.explain_grids_manipulation()
        self.clear_stage()

        # --- NEW SECTION 3: ANCHORS & LAYERS ---
        self.explain_anchors_layers()
        self.clear_stage()

        # --- ORIGINAL FINAL EXAMPLES ---
        self.run_histogram_example()
        self.clear_stage()
        
        self.run_solar_system_example()
        
        self.update_explanation("50+ Layout APIs Covered!", flicker=False)
        self.wait(3)

    # ==========================================
    # ORIGINAL SECTIONS (With Enhanced Visuals)
    # ==========================================

    def explain_groups(self):
        self.update_explanation("VGroup acts as a single container. Styles cascade to children.")
        code_text = """
        # 1. Create Items
        s = Square(color=WHITE)
        c = Circle(color=WHITE)
        
        # 2. Init VGroup
        g = VGroup(s, c)
        
        # 3. Cascade Style
        g.set_color(RED)
        g.set_opacity(0.5)
        
        # 4. Layout as one unit
        g.arrange(RIGHT)
        """
        lines = self.show_code_manual(code_text)
        
        # 1. Create Items
        s = Square(side_length=1, color=WHITE).shift(LEFT*1.5)
        c = Circle(radius=0.5, color=WHITE).shift(RIGHT*0.5)
        self.highlight_manual_line(lines, 2)
        self.play(Create(s), Create(c))
        self.wait(0.5)
        
        # 2. Init VGroup
        self.highlight_manual_line(lines, 6)
        group = VGroup(s, c)
        
        # VISUAL AID: Show the VGroup container
        # FIX: Removed invalid 'stroke_dash_length' argument
        group_outline = SurroundingRectangle(group, color=YELLOW, buff=0.1)
        label = Text("VGroup", font_size=16, color=YELLOW).next_to(group_outline, UP)
        self.play(Create(group_outline), Write(label))
        self.wait(0.5)
        
        # 3. Cascade Style
        self.highlight_manual_line(lines, 9)
        self.play(
            group.animate.set_color(RED).set_opacity(0.5),
            group_outline.animate.set_color(RED), # Sync visual aid
            run_time=1
        )
        self.wait(0.5)
        
        # 4. Layout as one unit
        self.highlight_manual_line(lines, 13)
        self.update_explanation("Ordering children relative to each other...")
        self.play(
            group.animate.arrange(RIGHT, buff=0.5),
            FadeOut(group_outline), FadeOut(label) # Refresh outline after move
        )
        
        # Re-draw outline to show new shape
        # FIX: Removed invalid 'stroke_dash_length' argument
        group_outline = SurroundingRectangle(group, color=YELLOW, buff=0.1)
        self.play(Create(group_outline))
        self.wait(1)

    def explain_arrange(self):
        self.update_explanation("Arrange shifts objects. 'aligned_edge' defines the baseline.")
        s_small = Square(side_length=0.5, color=BLUE, fill_opacity=1)
        s_tall = Rectangle(height=2, width=0.5, color=BLUE, fill_opacity=1)
        s_med = Square(side_length=1.0, color=BLUE, fill_opacity=1)
        g = VGroup(s_small, s_tall, s_med).move_to(self.stage_center)
        self.add(g)
        
        code_text = """
        # 1. Default (Center Align)
        g.arrange(RIGHT)
        
        # 2. Align Bottoms
        g.arrange(RIGHT, aligned_edge=DOWN)
        
        # 3. Align Tops
        g.arrange(RIGHT, aligned_edge=UP)
        """
        lines = self.show_code_manual(code_text)
        
        # 1. Default
        self.highlight_manual_line(lines, 2)
        self.play(g.animate.arrange(RIGHT, buff=0.5))
        
        # Visual Aid: Center Line
        line = DashedLine(start=g.get_left(), end=g.get_right(), color=YELLOW)
        self.play(Create(line), run_time=0.5)
        self.play(FadeOut(line), run_time=0.2)
        self.wait(0.5)
        
        # 2. Align Bottoms
        self.highlight_manual_line(lines, 5)
        self.play(g.animate.arrange(RIGHT, buff=0.5, aligned_edge=DOWN))
        
        # Visual Aid: Bottom Line
        line = DashedLine(start=g.get_corner(DL)+LEFT, end=g.get_corner(DR)+RIGHT, color=YELLOW)
        self.add(line)
        self.play(Wiggle(line))
        self.play(FadeOut(line))
        self.wait(0.5)
        
        # 3. Align Tops
        self.highlight_manual_line(lines, 8)
        self.play(g.animate.arrange(RIGHT, buff=0.5, aligned_edge=UP))
        
        # Visual Aid: Top Line
        line = DashedLine(start=g.get_corner(UL)+LEFT, end=g.get_corner(UR)+RIGHT, color=YELLOW)
        self.play(Create(line))
        self.play(FadeOut(line))
        self.wait(1)
        
        self.play(FadeOut(g))

    def explain_alignment_matchers(self):
        self.update_explanation("Precise alignment using match_x, match_y, and align_to.")
        ref = Rectangle(width=4, height=3, color=GRAY, fill_opacity=0.2).move_to(self.stage_center)
        ref_label = Text("Reference", font_size=16, color=GRAY).next_to(ref, UP)
        dot = Dot(color=YELLOW, radius=0.2).move_to(self.stage_center)
        self.add(ref, ref_label, dot)
        
        code_text = """
        # 1. Match Top Edge (Inside)
        dot.align_to(ref, UP)
        
        # 2. Match Width
        line.match_width(ref)
        
        # 3. Match X-Axis (Center)
        dot.match_x(ref_corner)
        """
        lines = self.show_code_manual(code_text)
        
        # 1. Match Top Edge
        self.highlight_manual_line(lines, 2)
        
        # Visual Aid: Show target edge
        target_line = DashedLine(ref.get_corner(UL), ref.get_corner(UR), color=YELLOW)
        self.play(Create(target_line))
        self.play(dot.animate.align_to(ref, UP))
        self.play(FadeOut(target_line))
        self.wait(1)
        
        # 2. Match Width
        self.highlight_manual_line(lines, 5)
        line = Line(LEFT, RIGHT, color=RED, stroke_width=8).next_to(ref, DOWN, buff=0.1).scale(0.2)
        self.add(line)
        self.play(line.animate.match_width(ref))
        
        # Visual Aid: Width Brackets
        brace = Brace(ref, DOWN)
        self.play(Create(brace))
        self.play(FadeOut(brace))
        self.wait(1)
        
        # 3. Match X-Axis
        self.highlight_manual_line(lines, 8)
        self.play(dot.animate.move_to(self.stage_center + LEFT*2))
        
        target_corner = ref.get_corner(UR)
        corner_dot = Dot(target_corner, color=RED, radius=0.1)
        self.play(Create(corner_dot))
        
        # Visual Aid: Vertical line for X alignment
        v_line = DashedLine(target_corner + UP, target_corner + DOWN*3, color=RED)
        self.play(Create(v_line))
        
        self.play(dot.animate.match_x(Point(target_corner)))
        self.play(FadeOut(v_line), FadeOut(corner_dot))
        self.wait(1)
        
        self.play(FadeOut(ref, dot, line, ref_label))

    # ==========================================
    # NEW SECTIONS (With Enhanced Visuals)
    # ==========================================

    def explain_sizing_stretch(self):
        self.update_explanation("Sizing APIs: set_width vs stretching distortion.")
        
        # Setup: A target box and a circle
        target_box = Rectangle(width=3, height=1.5, color=GREEN, fill_opacity=0.3).move_to(self.stage_center)
        c = Circle(radius=1, color=RED, fill_opacity=0.5).to_edge(UP)
        self.add(target_box, c)

        code_text = """
        # 1. Scale to Width (Keep Aspect)
        obj.set_width(2.0)
        
        # 2. Stretch Height (Distort)
        obj.stretch_to_fit_height(1.5)
        
        # 3. Match Another's Height
        obj.match_height(box)
        """
        lines = self.show_code_manual(code_text)

        # 1. Set Width
        self.highlight_manual_line(lines, 2)
        self.update_explanation("set_width() scales uniformly. Circle remains circular.")
        self.play(c.animate.set_width(2.0).move_to(self.stage_center + LEFT*0.5 + UP*0.5))
        self.wait(1)

        # 2. Stretch Height
        self.highlight_manual_line(lines, 5)
        self.update_explanation("stretch_to_fit...() distorts the shape.")
        
        # Visual Aid: Leave a ghost behind
        ghost = c.copy().set_opacity(0.2).set_stroke(opacity=0.2)
        self.add(ghost)
        
        self.play(c.animate.stretch_to_fit_height(0.5))
        self.play(Indicate(c, color=RED))
        self.play(FadeOut(ghost))
        self.wait(1)

        # 3. Match Height
        self.highlight_manual_line(lines, 8)
        self.update_explanation("match_height() scales to equal the target's height.")
        
        # Visual Aid: Extension lines from box
        l1 = DashedLine(target_box.get_corner(UR), target_box.get_corner(UR)+RIGHT*3, color=GREEN)
        l2 = DashedLine(target_box.get_corner(DR), target_box.get_corner(DR)+RIGHT*3, color=GREEN)
        self.play(Create(l1), Create(l2))
        
        self.play(c.animate.match_height(target_box))
        self.play(FadeOut(l1), FadeOut(l2))
        self.wait(1)
        
        self.play(FadeOut(c, target_box))

    def explain_grids_manipulation(self):
        self.update_explanation("Grids & Dynamic Groups: arrange_in_grid, add, remove.")
        
        dots = VGroup(*[Dot(radius=0.2, color=BLUE) for _ in range(9)])
        dots.move_to(self.stage_center)
        self.add(dots)

        code_text = """
        # 1. Grid Layout
        g.arrange_in_grid(rows=3, cols=3)
        
        # 2. Dynamic Add
        g.add(new_obj)
        
        # 3. Dynamic Remove
        g.remove(g[0])
        """
        lines = self.show_code_manual(code_text)

        # 1. Arrange in Grid
        self.highlight_manual_line(lines, 2)
        self.play(dots.animate.arrange_in_grid(rows=3, cols=3, buff=0.5))
        
        # Visual Aid: Flash rows
        for i in range(3):
            row = VGroup(dots[3*i], dots[3*i+1], dots[3*i+2])
            self.play(row.animate.set_color(TEAL), run_time=0.2)
            self.play(row.animate.set_color(BLUE), run_time=0.2)
        self.wait(0.5)

        # 2. Dynamic Add
        self.highlight_manual_line(lines, 5)
        new_dot = Dot(color=RED).move_to(self.stage_center + RIGHT*3)
        self.play(FadeIn(new_dot))
        
        # Logically add to group, then re-arrange to show effect
        dots.add(new_dot) 
        self.update_explanation("Added red dot to group. Re-arranging flow...")
        self.play(dots.animate.arrange_in_grid(rows=2, cols=5, buff=0.3))
        
        # Highlight the new member's position
        self.play(Indicate(new_dot, scale_factor=1.5))
        self.wait(1)

        # 3. Dynamic Remove
        self.highlight_manual_line(lines, 8)
        to_remove = dots[0] # Top left usually
        
        self.play(to_remove.animate.set_color(RED))
        dots.remove(to_remove)
        
        self.update_explanation("Removing item 0. Grid reflows automatically.")
        self.play(
            FadeOut(to_remove), 
            dots.animate.arrange_in_grid(rows=3, cols=3, buff=0.3)
        )
        self.wait(1)

        self.play(FadeOut(dots))

    def explain_anchors_layers(self):
        self.update_explanation("Anchors (get_corner) & Layers (z_index).")
        
        box = Square(side_length=3, color=WHITE).move_to(self.stage_center)
        dot = Dot(color=YELLOW)
        self.add(box, dot)

        code_text = """
        # 1. Get Corner (UR = UpRight)
        pos = box.get_corner(UR)
        
        # 2. Get Edge Center
        pos = box.get_edge_center(LEFT)
        
        # 3. Z-Index (Layering)
        obj.set_z_index(10)
        """
        lines = self.show_code_manual(code_text)

        # 1. Get Corner
        self.highlight_manual_line(lines, 2)
        corner_pos = box.get_corner(UR)
        
        # Visual Aid: Flash the point
        flash_dot = Dot(corner_pos, color=RED).scale(1.5)
        self.play(FadeIn(flash_dot, scale=0.5), run_time=0.3)
        self.play(FadeOut(flash_dot), run_time=0.3)
        
        self.play(dot.animate.move_to(corner_pos))
        self.wait(0.5)

        # 2. Get Edge Center
        self.highlight_manual_line(lines, 5)
        edge_pos = box.get_edge_center(LEFT)
        
        # Visual Aid: Flash the edge
        flash_line = Line(box.get_corner(UL), box.get_corner(DL), color=RED, stroke_width=6)
        self.play(Create(flash_line), run_time=0.3)
        self.play(FadeOut(flash_line), run_time=0.3)
        
        self.play(dot.animate.move_to(edge_pos))
        self.wait(0.5)

        # 3. Z-Index and Surrounding
        self.highlight_manual_line(lines, 8)
        
        # Create a filled square that covers the dot
        cover = Square(side_length=1, fill_color=RED, fill_opacity=1).move_to(edge_pos)
        self.play(FadeIn(cover)) 
        self.update_explanation("Red square covers dot. Default Z is 0.")
        self.wait(1)
        
        self.update_explanation("Setting dot z_index=10 brings it to front.")
        dot.set_z_index(10) # API Usage
        
        # Trigger visual update with a pop
        self.play(
            dot.animate.scale(1.5), 
            rate_func=wiggle
        )
        self.play(dot.animate.scale(1/1.5))
        self.wait(1)

        # Bonus: Surrounding Rectangle
        surr = SurroundingRectangle(box, color=TEAL, buff=0.1)
        self.play(Create(surr))
        self.wait(1)
        
        self.play(FadeOut(box, dot, cover, surr))

    # ==========================================
    # FINAL EXAMPLES
    # ==========================================

    def run_histogram_example(self):
        self.update_explanation("Ex 1: Dynamic Histogram (Groups + Alignment + Matching)")
        code_text = """
        # 1. Create Bars
        bars = VGroup(...)
        
        # 2. Arrange Bottoms
        bars.arrange(RIGHT, aligned_edge=DOWN)
        
        # 3. Add Axes
        line.match_width(bars)
        line.next_to(bars, DOWN, buff=0)
        """
        lines = self.show_code_manual(code_text)
        values = [1.5, 2.5, 1.0, 3.0, 2.0]
        bars = VGroup(*[Rectangle(height=v, width=0.6, fill_opacity=0.8, color=TEAL) for v in values])
        
        # Scramble positions initially
        for b in bars: b.move_to(self.stage_center + [np.random.uniform(-1,1), np.random.uniform(-1,1), 0])
        self.play(FadeIn(bars))
        
        self.highlight_manual_line(lines, 5)
        self.play(bars.animate.arrange(RIGHT, buff=0.2, aligned_edge=DOWN))
        self.wait(0.5)
        
        self.highlight_manual_line(lines, 8)
        axis = Line(ORIGIN, RIGHT, color=WHITE)
        
        # Visual Aid: Show width matching logic
        self.play(axis.animate.match_width(bars).scale(1.2).next_to(bars, DOWN, buff=0))
        
        labels = VGroup()
        for i, bar in enumerate(bars):
            lbl = Text(str(values[i]), font_size=14).next_to(bar, UP, buff=0.1)
            labels.add(lbl)
        self.play(Write(labels))
        self.wait(2)

    def run_solar_system_example(self):
        self.update_explanation("Ex 2: Nested Groups & Rotation (Solar System)")
        code_text = """
        # 1. Nest Groups
        system = VGroup(sun, planet, moon)
        
        # 2. Relative Layout
        planet.next_to(sun, RIGHT, buff=2)
        moon.next_to(planet, RIGHT, buff=0.5)
        
        # 3. Rotate Group
        system.rotate(angle, about_point=SUN)
        """
        lines = self.show_code_manual(code_text)
        sun = Dot(radius=0.4, color=YELLOW).move_to(self.stage_center)
        earth = Dot(radius=0.15, color=BLUE)
        moon = Dot(radius=0.05, color=GRAY)
        self.play(FadeIn(sun), FadeIn(earth), FadeIn(moon))
        
        self.highlight_manual_line(lines, 5)
        
        # Show arrows for "next_to" relation
        arrow1 = Arrow(sun.get_right(), sun.get_right()+RIGHT*1.5, buff=0, color=WHITE, stroke_width=2)
        self.play(Create(arrow1))
        self.play(earth.animate.next_to(sun, RIGHT, buff=1.5), FadeOut(arrow1))
        
        self.play(moon.animate.next_to(earth, RIGHT, buff=0.3))
        self.wait(0.5)
        
        self.highlight_manual_line(lines, 9)
        earth_moon = VGroup(earth, moon)
        
        # Add visual trails
        trace_earth = TracedPath(earth.get_center, stroke_color=BLUE, stroke_opacity=0.5, dissipating_time=2)
        trace_moon = TracedPath(moon.get_center, stroke_color=GRAY, stroke_opacity=0.5, dissipating_time=1)
        self.add(trace_earth, trace_moon)
        
        self.play(
            Rotate(earth_moon, angle=2*PI, about_point=sun.get_center(), run_time=4, rate_func=linear), 
            Rotate(moon, angle=6*PI, about_point=earth.get_center(), run_time=4, rate_func=linear)
        )
        self.wait(1)

    # ==========================================
    # HELPER METHODS (SCAFFOLDING)
    # ==========================================
    def get_grid(self):
        plane = NumberPlane(
            x_range=[-4, 4, 1], y_range=[-3, 3, 1], 
            x_length=6, y_length=4.2, 
            background_line_style={"stroke_color": TEAL, "stroke_width": 1, "stroke_opacity": 0.3},
            axis_config={"stroke_color": TEAL, "stroke_width": 1, "include_numbers": False} 
        ).move_to(self.stage_center)
        return plane

    def setup_scaffolding(self, title_text):
        self.title = Text(title_text, font_size=24).to_edge(UP, buff=0.2)
        div_line = Line(LEFT*6, RIGHT*6, color=WHITE, stroke_width=2).next_to(self.title, DOWN, buff=0.2)
        self.code_bg = RoundedRectangle(corner_radius=0.2, width=6, height=4.5, fill_color="#222222", fill_opacity=1, stroke_color=GRAY)
        self.code_bg.next_to(div_line, DOWN, buff=0.5).to_edge(LEFT, buff=0.5)
        code_label = Text("Source Code", font_size=18, color=GRAY).next_to(self.code_bg, UP, buff=0.1)
        self.stage_bg = RoundedRectangle(corner_radius=0.2, width=6.5, height=4.5, fill_color="#000000", fill_opacity=1, stroke_color=WHITE)
        self.stage_bg.match_y(self.code_bg).to_edge(RIGHT, buff=0.5)
        self.stage_center = self.stage_bg.get_center()
        stage_label = Text("Output", font_size=18, color=GRAY).next_to(self.stage_bg, UP, buff=0.1)
        self.expl_bg = Rectangle(width=13, height=1.5, fill_color="#111111", fill_opacity=1, stroke_width=0)
        self.expl_bg.to_edge(DOWN, buff=0.1)
        self.stage_grid = self.get_grid()
        self.add(self.title, div_line, self.code_bg, code_label, self.stage_bg, stage_label, self.expl_bg, self.stage_grid)

    def update_explanation(self, text, flicker=True):
        if hasattr(self, 'current_expl'): self.remove(self.current_expl)
        
        # IMPROVEMENT: Text wrapping to ensure it fits the screen width (approx 65 chars)
        wrapped_text = "\n".join(textwrap.wrap(text, width=65))
        
        self.current_expl = Text(
            wrapped_text, 
            font_size=20, 
            line_spacing=1.2, 
            t2c={"VGroup": YELLOW, "arrange": YELLOW, "next_to": YELLOW, "align_to": YELLOW, "set_width": YELLOW, "grid": YELLOW, "z_index": YELLOW}
        ).move_to(self.expl_bg)
        
        if flicker: self.play(Write(self.current_expl), run_time=0.5)
        else: self.add(self.current_expl)

    def show_code_manual(self, code_str):
        if hasattr(self, 'current_code_group'): self.remove(self.current_code_group)
        if hasattr(self, 'current_highlighter'): self.remove(self.current_highlighter)
        lines = textwrap.dedent(code_str).strip().split('\n')
        text_group = VGroup()
        start_x = self.code_bg.get_left()[0] + 0.3
        start_y = self.code_bg.get_top()[1] - 0.3
        for i, line in enumerate(lines):
            c = GOLD if "self.play" in line or "self.add" in line else (GRAY_C if line.strip().startswith("#") else WHITE)
            t = Text(line, font="Monospace", font_size=16, color=c)
            t.move_to([start_x, start_y - (i * 0.4), 0], aligned_edge=LEFT)
            text_group.add(t)
        self.current_code_group = text_group
        self.play(FadeIn(text_group), run_time=0.5)
        return text_group

    def highlight_manual_line(self, code_group, line_num):
        idx = line_num - 1
        if idx >= len(code_group): return
        surround = SurroundingRectangle(code_group[idx], color=YELLOW, buff=0.05, stroke_width=2)
        if hasattr(self, 'current_highlighter'): self.play(Transform(self.current_highlighter, surround), run_time=0.2)
        else: self.current_highlighter = surround; self.play(Create(surround), run_time=0.2)

    def clear_stage(self):
        static = [self.title, self.code_bg, self.stage_bg, self.expl_bg, self.current_expl, getattr(self, 'current_code_group', None), getattr(self, 'current_highlighter', None), getattr(self, 'stage_grid', None)]
        to_rem = [m for m in self.mobjects if m not in static and m is not None]
        if to_rem: self.play(FadeOut(*to_rem), run_time=0.5)

%manim -qk -v warning LayoutDeepDiveExtended